---
layout: post
title: GSpread
subtitle: Exercícios e Referências
tags: [python, pycharm, jupyter, package, gspread]
image: /img/posts/gspread_icon.png
bigimg: /img/posts/gspread_big.png
gh-repo: michelmetran/package_gspread
gh-badge: [follow, star, watch, fork]
comments: true

---

O <a title="Link do GSpread" href="https://gspread.readthedocs.io/en/latest/api.html" target="_blank">**_GSpread_**</a> é um pacote que possibilita a edição e obtenção de dados das planilhas do  _Google SpreadSheet_, utilizando _Python_. Ou seja, é possível manipular os dados utiliznado toda uma variedade de _packages_ do _Python_ e inseri-los em _Google Spreadsheets_.

Outro pacote relevante para trabalhar junto é o <a title="Link do df2gspread" href="https://df2gspread.readthedocs.io/en/latest/index.html" target="_blank">**_df2gspread_**</a>, que lê a tabela no formado do **_Pandas_**.

<br>

{: .box-warning}
**Aviso:** Esse _post_ tem a finalidade de mostrar os comandos básicos e me deixar com uma "cola" rápida para meu uso cotidiano. Todas os códigos são exemplificativos e podem/devem ser alterados, indicando o nome dos arquivos e diretórios corretamente.

{: .box-note}
**Nota:** É possível acessar esse _post_ em formato <a title="Link do Folium" href="https://github.com/michelmetran/package_gspread/raw/master/docs/gspread.pdf" target="_blank">**_pdf_**</a>, diretamente por meio do <a title="Link do Repositório" href="https://github.com/michelmetran/package_gspread" target="_blank">**repositório do GitHub**</a> ou ainda, de maneira interativa, usando o [![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/michelmetran/package_gspread/master).

<br>

# Importando Bibliotecas

As bibliotecas básicas, ou _packages_, necessárias para criação do mapa são:
- O **_GSpread_**, que é a biblioteca que lê e edita as tabelas do Google;
- O **df2gspread** é uma biblioteca que, junto com o **_GSpread_**, lê e escreve as tabelas no formato do **_Pandas_**.

In [159]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

from df2gspread import df2gspread as d2g
from df2gspread import gspread2df as g2d

<br>

# Autenticando no _Google Spreadsheet_

Códigos necessários para "entrar" em modo de edição de uma planilha, a partir do _python_.

In [160]:
# Escopo Utilizado
scope = ['https://spreadsheets.google.com/feeds']

api_file = 'python-gspread-credentials.json'

# Lê a Credencial para Autenticação
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    os.path.join('..', 'codes/vault', api_file),
    scope)

# Autenticação, de fato
gc = gspread.authorize(credentials)

Escolhe a planilha e aba a ser editada

In [161]:
# Abre a Planilha
wkb = gc.open_by_key('1bRwjoieInaRkmoyvlisMtDi-2kZ-5CvyKEQ2V_Xk1U8')    # Pelo ID da planilha
#wkb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1bRwjoieInaRkmoyvlisMtDi-2kZ-5CvyKEQ2V_Xk1U8/edit#gid=0')    # Pela URL da planilha
#wkb = gc.open('Teste Python')               # Pelo nome da planilha

# Seleciona a primeira página da planilha
wks = wkb.get_worksheet(0)             # Pela ordem, começando com 0
#wks = wkb.worksheet('January')        # Pelo nome da aba

# Escrevendo Dados

## Célula Individual

In [162]:
# Pela notação padrão do Google Spreadshhet
wks.update_acell('A1', 'Dia e Hora')

{'spreadsheetId': '1bRwjoieInaRkmoyvlisMtDi-2kZ-5CvyKEQ2V_Xk1U8',
 'updatedRange': "'Página1'!A1",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [163]:
# Pela notação de coordenadas
wks.update_cell(1, 2, 'Bingo!')

{'spreadsheetId': '1bRwjoieInaRkmoyvlisMtDi-2kZ-5CvyKEQ2V_Xk1U8',
 'updatedRange': "'Página1'!B1",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [164]:
%run '~/Documents/SourceCode/codes/files/str_today.py'
wks.update_acell('A2', srt_today())

{'spreadsheetId': '1bRwjoieInaRkmoyvlisMtDi-2kZ-5CvyKEQ2V_Xk1U8',
 'updatedRange': "'Página1'!A2",
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

## Conjunto de Células

In [165]:
# Cabeçalho
wks.update_acell('A1', 'Estado')
wks.update_acell('B1', 'Capital')

# Dicionário com os estados e as capitais
capitais = {'Paraíba': 'João Pessoa', 'Santa Catarina': 'Florianópolis', 'São Paulo': 'São Paulo'}

# Contador de colunas e celulas
column = 1
row = 2

for estado, capital in capitais.items():
    # Atualiza a celula 2 da coluna 1 com o nome do estado 
    wks.update_cell(row, column, estado)
    
    # A coluna agora é a B
    column = 2
    
    # Atualiza a celula 2 da coluna 2 com o nome da capital
    wks.update_cell(row, column, capital)
    
    # A coluna agora é a A
    column = 1
    
    # Acrescenta mais um valor no numero da celula
    row += 1

In [166]:
# Atualiza um Conjunto de Células iguais
cell_list = wks.range('A1:C7')

for cell in cell_list:
    cell.value = '00'

# Update in batch
wks.update_cells(cell_list)

{'spreadsheetId': '1bRwjoieInaRkmoyvlisMtDi-2kZ-5CvyKEQ2V_Xk1U8',
 'updatedRange': "'Página1'!A1:C7",
 'updatedRows': 7,
 'updatedColumns': 3,
 'updatedCells': 21}

## Arquivo para _Google Spreadsheet_

In [167]:
# Lendo e filtrando dados
empresas = pd.read_csv('data/empresas.xz')
empresas = empresas[empresas['state'] == 'SP']
empresas = empresas[empresas['city'] == 'SANTOS']

empresas.dtypes

name                           object
situation                      object
neighborhood                   object
address                        object
number                         object
zip_code                       object
city                           object
state                          object
cnpj                           object
status                         object
additional_address_details     object
main_activity                  object
latitude                      float64
longitude                     float64
dtype: object

In [168]:
from df2gspread import df2gspread as d2g

d2g.upload(empresas,
           gfile='1bRwjoieInaRkmoyvlisMtDi-2kZ-5CvyKEQ2V_Xk1U8',
           wks_name='Página1',
           start_cell='A1',
           credentials=credentials,           
           col_names=True,
           row_names=False,           
           clean=False)

<Worksheet 'Página1' id:0>

In [169]:
wks.title

'Página1'

# Lendo Dados

## Célula Individual

In [106]:
# Pega o valor específico de uma célula
val = wks.cell(1,1).value
print(val)

name


In [108]:
val = wks.acell('A3').value
print(val)

MARCIO CESAR FRANCISQUINE SANTOS - ME


In [98]:
# Seleciona todos os dados de uma coluna
val = wks.col_values(1)
print(val)

['name', 'ALMEIDA & RODRIGUES MANUTENCAO EM INFORMATICA LTDA - ME', 'MARCIO CESAR FRANCISQUINE SANTOS - ME', 'R P LOPES FONSECA', 'AUTO POSTO NOVO MILENIO LTDA', "CHURRASCARIA E PIZZARIA MAXIM'S LTDA - ME", 'HOTEL DE TURISMO PARQUE BALNEARIO LTDA', 'AUTO POSTO BRUMAR LTDA', 'ATLANTICO GOLDEN APART HOTEL LTDA.', 'COOPERATIVA DOS CONDUTORES AUTONOMOS DE VEICULOS RODOVIARIOS DA BAIXADA SANTISTA', 'R.P.W. - EQUIPAMENTOS PARA ESCRITORIO LTDA - EPP', 'ASSOC. DOS COND. AUTON. DE TAXIS DE SANTOS RADIO TAXI', 'POSTO SANTO SOL LTDA', 'COMPANHIA BRASILEIRA DE DISTRIBUICAO', 'CAR RENTAL SYSTEMS DO BRASIL LOCACAO DE VEICULOS LTDA', 'ADALBERTO NASCIMENTO DOS SANTOS 00243924801', 'CODY MATERIAIS E SERVICOS LTDA', 'ALBA ESTACIONAMENTO LTDA - ME', 'AUTO POSTO FORMULA 3 LTDA', 'CLEAN CAR SUPER LAVAGEM AUTOMATICA E COMERCIO LTDA', 'AUTO POSTO JARDIM ANCHIETA LIMITADA', 'WORK INTERNET LTDA - ME', 'EDITORA ECOTURISMO LTDA. - ME', 'AUTO POSTO CANAL OK LTDA', 'MUNICIPIO DE SANTOS', 'A TRIBUNA DE SANTOS JORNA

In [99]:
# Seleciona todos os dados de uma linha
val = wks.row_values(1)
print(val)

['name', 'situation', 'neighborhood', 'address', 'number', 'zip_code', 'city', 'state', 'cnpj', 'status', 'additional_address_details', 'main_activity', 'latitude', 'longitude']


In [100]:
# Procurando uma localização específica
cell = wks.find('CENTRO')
print(f'Encontrado na celula {cell.row} coluna {cell.col}')

Encontrado na celula 2 coluna 3


In [102]:
# Gera uma lista de listas
list_of_lists = wks.get_all_values()
list_of_lists

[['name',
  'situation',
  'neighborhood',
  'address',
  'number',
  'zip_code',
  'city',
  'state',
  'cnpj',
  'status',
  'additional_address_details',
  'main_activity',
  'latitude',
  'longitude'],
 ['ALMEIDA & RODRIGUES MANUTENCAO EM INFORMATICA LTDA - ME',
  'ATIVA',
  'CENTRO',
  'R AMADOR BUENO',
  '171',
  '11.013-151',
  'SANTOS',
  'SP',
  '13.295.386/0001-58',
  'OK',
  'SALA 69',
  'Reparação e manutenção de computadores e de equipamentos periféricos',
  '-23.935971600000002',
  '-46.3272156'],
 ['MARCIO CESAR FRANCISQUINE SANTOS - ME',
  'ATIVA',
  'MARAPE',
  'AV SENADOR PINHEIRO MACHADO',
  '856',
  '11.075-002',
  'SANTOS',
  'SP',
  '18.950.028/0001-55',
  'OK',
  'nan',
  'Restaurantes e similares',
  '-23.961729300000002',
  '-46.3454754'],
 ['R P LOPES FONSECA',
  'ATIVA',
  'PONTA DA PRAIA',
  'PC CORACAO DE MARIA',
  '23',
  '11.030-280',
  'SANTOS',
  'SP',
  '58.132.036/0001-09',
  'OK',
  'nan',
  'Comércio varejista de combustíveis para veículos automotor

## _Google Spreadsheet_ para arquivo

In [172]:
from df2gspread import gspread2df as g2d

tab = g2d.download(gfile='1bRwjoieInaRkmoyvlisMtDi-2kZ-5CvyKEQ2V_Xk1U8',
                   wks_name='Página1',
                   start_cell='A1',
                   credentials=credentials,
                   col_names=True,
                   row_names=False)

tab.head(2)

,name,situation,neighborhood,address,number,zip_code,city,state,cnpj,status,additional_address_details,main_activity,latitude,longitude
0,ALMEIDA & RODRIGUES MANUTENCAO EM INFORMATICA ...,ATIVA,CENTRO,R AMADOR BUENO,171,11.013-151,SANTOS,SP,13.295.386/0001-58,OK,SALA 69,Reparação e manutenção de computadores e de eq...,-23.935971600000002,-46.3272156
1,MARCIO CESAR FRANCISQUINE SANTOS - ME,ATIVA,MARAPE,AV SENADOR PINHEIRO MACHADO,856,11.075-002,SANTOS,SP,18.950.028/0001-55,OK,nan,Restaurantes e similares,-23.961729300000002,-46.3454754


In [173]:
# Lixo, com o df2gspread passou a ser desnecessário esse código
import pandas as pd

data = wks.get_all_values()
#data = wks.get_all_records(empty2zero=False, head=4, default_blank='',allow_underscores_in_numeric_literals=False)
headers = data.pop(0)

tab = pd.DataFrame(data, columns=headers)

tab.head(2)

,name,situation,neighborhood,address,number,zip_code,city,state,cnpj,status,additional_address_details,main_activity,latitude,longitude
0,ALMEIDA & RODRIGUES MANUTENCAO EM INFORMATICA ...,ATIVA,CENTRO,R AMADOR BUENO,171,11.013-151,SANTOS,SP,13.295.386/0001-58,OK,SALA 69,Reparação e manutenção de computadores e de eq...,-23.935971600000002,-46.3272156
1,MARCIO CESAR FRANCISQUINE SANTOS - ME,ATIVA,MARAPE,AV SENADOR PINHEIRO MACHADO,856,11.075-002,SANTOS,SP,18.950.028/0001-55,OK,nan,Restaurantes e similares,-23.961729300000002,-46.3454754


<br>

# Referências
- Toda introdução está em https://www.linkedin.com/pulse/manipulando-planilhas-do-google-usando-python-renan-pessoa
- Um guia de referências https://gspread.readthedocs.io/en/latest/user-guide.html
- https://df2gspread.readthedocs.io/en/latest/index.html

___
<br>

# Exportando o _Juptyter Notebook_ para outros formatos
O arquivo _.ipynb_ pode ser exportado em formatos diversos. Abaixo carrego uma função que escrevi para facilitar o processo de exportação do arquivo em diferentes locais do PC para, posteriormente, atualizar os repositórios contidos no <a title="Link do GitHub" href="https://github.com/michelmetran" target="_blank">_GitHub_</a>.

In [14]:
# %load '~/Documents/SourceCode/codes/files/export_jupyter.py'
def export_jupyter(path, extensions=['html', 'markdown', 'latex', 'pdf', 'python'], today=True):
    """
    Export .ipynb file to others formats
    :return: File in other formats
    """
    # Import Packages
    import os
    import datetime

    # Data
    timestamp = datetime.datetime.now()
    srt_today = (str(timestamp.year) + '-' +
                 str(f"{timestamp.month:02d}") + '-' +
                 str(f"{timestamp.day:02d}"))

    # Extensions
    for extension in extensions:
        if today==True:
            os.system('jupyter nbconvert --to {} {} --output {}'.
                      format(extension, get_jupyternotebook_name(),
                             os.path.join(path, srt_today+'-'+get_jupyternotebook_name().split('.')[0])))
            print('Arquivo {} exportado corretamente para o formato {} usando prefixo da data.'.
                  format(get_jupyternotebook_name(), extension))

        else:
            os.system('jupyter nbconvert --to {} {} --output {}'.
                      format(extension, get_jupyternotebook_name(),
                             os.path.join(path, get_jupyternotebook_name().split('.')[0])))
            print('Arquivo {} exportado corretamente para o formato {} sem usar prefixo da data.'.
                  format(get_jupyternotebook_name(), extension))


In [21]:
# %load '~/Documents/SourceCode/codes/files/get_jupyternotebook_name.py'
def get_jupyternotebook_name():
    """
    Returns the name of the current notebook as a string
    From https://mail.scipy.org/pipermail/ipython-dev/2014-June/014096.html
    :return: Returns the name of the current notebook as a string
    """
    # Import Packages
    from IPython.core.display import Javascript
    from IPython.display import display

    display(Javascript('IPython.notebook.kernel.execute("theNotebook = " + \
    "\'"+IPython.notebook.notebook_name+"\'");'))

    # Result
    return theNotebook


Com as funções para exportar o _Jupyter Notebook_ e para obter o nome do arquivo _.ipynb_ carregadas, basta exportar o arquivo, inicialmente para a pasta _docs_ dentro do projeto e também, visando atualizar os _posts_ do site, para a respectiva pasta.

In [22]:
export_jupyter('docs',['pdf'], False)
export_jupyter('/home/michel/Documents/SourceCode/michelmetran.github.io/_posts', ['markdown'], True)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Arquivo gspread.ipynb exportado corretamente para o formato pdf sem usar prefixo da data.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Arquivo gspread.ipynb exportado corretamente para o formato markdown usando prefixo da data.


<br>

# Atualizando Repositório do Projeto e do _site_
Após as exportações dos arquivos nos formatos necessários, basta atualizar o repositório diretamente pelo  _Jupyter Notebook_.
Abaixo é atualizado o repositório desse projeto específico, bem como a derivação desse projeto no <a title="Link do Folium" href="https://michelmetran.github.io/" target="_blank">**_site_**</a>.

In [23]:
%run '~/Documents/SourceCode/codes/git/update_github.py'

In [24]:
git_full('/home/michel/Documents/SourceCode/package_gspread', '.', 'Atualizando')
git_full('/home/michel/Documents/SourceCode/michelmetran.github.io', '.', 'Atualizando')

b'' b''
b'[master 392d752] Atualizando\n 2 files changed, 59 insertions(+), 50 deletions(-)\n delete mode 100644 python-gspread-credentials.json\n' b''
b'' b'To github.com:michelmetran/package_gspread.git\n   43e9936..392d752  master -> master\n'
Done!!
b'' b''
b'[master 4bed070] Atualizando\n 1 file changed, 34 insertions(+), 73 deletions(-)\n' b''
b'' b'To github.com:michelmetran/michelmetran.github.io.git\n   9d08db3..4bed070  master -> master\n'
Done!!
